In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f'Shape of X [N,C,H,W] : {X.shape}')
    print(f'shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N,C,H,W] : torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss : {loss:>7f} [{current:>5d}/{size:>5d}]')

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n')

In [9]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n---------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader, model,loss_fn)
print('Done!')

Epoch 1
---------------
loss : 2.306425 [    0/60000]
loss : 2.290571 [ 6400/60000]
loss : 2.276771 [12800/60000]
loss : 2.272496 [19200/60000]
loss : 2.236117 [25600/60000]
loss : 2.221013 [32000/60000]
loss : 2.227348 [38400/60000]
loss : 2.189948 [44800/60000]
loss : 2.183115 [51200/60000]
loss : 2.162508 [57600/60000]
Test Error : 
 Accuracy : 44.7%, Avg loss : 2.150388 

Epoch 2
---------------
loss : 2.158699 [    0/60000]
loss : 2.144531 [ 6400/60000]
loss : 2.092037 [12800/60000]
loss : 2.110528 [19200/60000]
loss : 2.042444 [25600/60000]
loss : 1.995211 [32000/60000]
loss : 2.016969 [38400/60000]
loss : 1.936551 [44800/60000]
loss : 1.934226 [51200/60000]
loss : 1.875403 [57600/60000]
Test Error : 
 Accuracy : 56.2%, Avg loss : 1.866827 

Epoch 3
---------------
loss : 1.899572 [    0/60000]
loss : 1.863710 [ 6400/60000]
loss : 1.750670 [12800/60000]
loss : 1.794290 [19200/60000]
loss : 1.681284 [25600/60000]
loss : 1.640844 [32000/60000]
loss : 1.656718 [38400/60000]
loss : 1